# Lesson 2 : LangGraph Components

In [ ]:
import re
import httpx
import os
from langchain_groq import ChatGroq
from dotenv import load_dotenv

load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

model_id = "openai/gpt-oss-120b" # 	qwen/qwen3-32b  openai/gpt-oss-120b llama-3.1-8b-instant
    
llm = ChatGroq(model=model_id,
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    verbose=1)

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
# from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [3]:
tool = TavilySearchResults(max_results=4) #increased number of results
print(type(tool))
print(tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


/var/folders/0z/6z_9lrtd6j3_gnlsyv9rb5b00000gn/T/ipykernel_39419/144666821.py:1: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tool = TavilySearchResults(max_results=4) #increased number of results


> If you are not familiar with python typing annotation, you can refer to the [python documents](https://docs.python.org/3/library/typing.html).

In [4]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

> Note: in `take_action` below, some logic was added to cover the case that the LLM returned a non-existent tool name. Even with function calling, LLMs can still occasionally hallucinate. Note that all that is done is instructing the LLM to try again! An advantage of an agentic organization.

In [8]:
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [9]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

abot = Agent(llm, [tool], system=prompt)

In [13]:
messages = [HumanMessage(content="What is the weather in sf?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather San Francisco'}, 'id': 'fc_079e1d51-c423-48eb-9e44-d15860f6753a', 'type': 'tool_call'}
Back to the model!
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco current temperature November 1 2025'}, 'id': 'fc_115868ab-20ee-4385-b042-4e2269eb29e5', 'type': 'tool_call'}
Back to the model!


In [20]:
print(result['messages'][-1].model_dump()['content'])

**Current weather in San Francisco (as of Saturday Nov 1 2025)**  

| Item | Details |
|------|---------|
| **Condition** | Mostly sunny with low‑level clouds and fog early in the morning; the fog clears as the day progresses. |
| **Temperature** | High around **67 °F (19 °C)**. Overnight low around **55 °F (13 °C)**. |
| **Wind** | Light and variable, generally from the west‑northwest at 5‑15 mph. |
| **Humidity** | Mid‑50 % to low‑60 % (typical for a foggy morning). |
| **Visibility** | Reduced in the early morning due to fog, improving to clear/mostly clear later in the day. |
| **Precipitation** | None reported for today; only a few scattered clouds expected tonight. |
| **Short‑term forecast** | <br>• **Tonight:** Mainly clear early, then patchy low clouds and fog. <br>• **Sunday (Nov 2):** Fog turning to sun, partly cloudy. <br>• **Monday (Nov 3):** Partly sunny, increasing cloudiness. |

*Sources:* AccuWeather’s “Today” page for San Francisco (high 67 °F, low 55 °F, fog early) a

In [22]:
messages = [HumanMessage(content="What is the weather in SF and LA?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather San Francisco'}, 'id': 'fc_2ace4188-f835-4f3c-9758-94caf2455154', 'type': 'tool_call'}
Back to the model!
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco current weather'}, 'id': 'fc_4724090d-3bed-4651-a503-b213830d28c4', 'type': 'tool_call'}
Back to the model!
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco weather now'}, 'id': 'fc_8edfefda-2f1c-47d2-a75c-4d302c943bcb', 'type': 'tool_call'}
Back to the model!
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Los Angeles current weather'}, 'id': 'fc_7b7bdb24-95f1-494d-b707-0b978ac612bf', 'type': 'tool_call'}
Back to the model!
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco weather now temperature'}, 'id': 'fc_f9e8de5e-77a9-4ff8-94f2-2a18c9b881fc', 'type': 'tool_call'}
Back to the model!
Calling: {'name': 'tavily_search_results_json', 'args

In [26]:
import pprint
print(result['messages'][-1].content)

**Weather – Nov 1 2025 (today)**  

| City | Current condition | Temperature (high / low) | Wind | Chance of rain |
|------|-------------------|---------------------------|------|----------------|
| **San Francisco (SF)** | Partly‑cloudy with occasional fog patches | **68 °F / 54 °F** (≈ 20 °C / 12 °C) | West 10‑15 mph (light) | 20‑30 % (scattered showers possible) |
| **Los Angeles (LA)** | Sunny, clear skies | **75 °F / 59 °F** (≈ 24 °C / 15 °C) | Light / variable (≈ 5‑10 mph) | < 5 % (dry) |

**What this means**

* **San Francisco** – The day will be mild and mostly sunny, but the typical coastal fog may drift in during the morning, giving way to partly‑cloudy skies by mid‑day. Temperatures stay in the upper‑60 °F range, with a light west wind. There’s a modest chance of an isolated shower or drizzle, especially near the coast.

* **Los Angeles** – Expect a classic Southern‑California November day: bright sunshine, warm daytime highs in the mid‑70 °F, and comfortable evenings in the

In [27]:
# Note, the query was modified to produce more consistent results. 
# Results may vary per run and over time as search information and models change.

query = "Who won the super bowl in 2024? In what state is the winning team headquarters located? \
What is the GDP of that state? Answer each question." 
messages = [HumanMessage(content=query)]

abot = Agent(llm, [tool], system=prompt)
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Missouri GDP 2023'}, 'id': 'fc_3c62a73c-5319-444b-ac1a-6f11eaa77e63', 'type': 'tool_call'}
Back to the model!
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Super Bowl 2024 winner Kansas City Chiefs'}, 'id': 'fc_6eec62ca-3564-4936-b201-dd4a116c08f6', 'type': 'tool_call'}
Back to the model!
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Missouri GDP 2024 real GDP $356.7 billion'}, 'id': 'fc_599ec43a-7a6c-40a6-9fe0-ce43442eef43', 'type': 'tool_call'}
Back to the model!


In [28]:
print(result['messages'][-1].content)

**1. Who won the Super Bowl in 2024?**  
- The **Kansas City Chiefs** won Super Bowl LVIII (played on February 11 2024), defeating the San Francisco 49ers 25‑22 in overtime【2†L1-L8】.

**2. In what state is the winning team’s headquarters located?**  
- The Kansas City Chiefs are headquartered in **Kansas City, Missouri** (the franchise’s home city and state).

**3. What is the GDP of that state?**  
- According to USAFacts, **Missouri’s real Gross Domestic Product (GDP) in 2024 was $356.7 billion**【3†L1-L7】.  
- (The U.S. Bureau of Economic Analysis also reports Missouri’s inflation‑adjusted GDP for 2024 at about **$353 billion**【3†L8-L12】.)

**Answer Summary**

| Question | Answer |
|----------|--------|
| Super Bowl 2024 winner | Kansas City Chiefs |
| State of the team’s headquarters | Missouri |
| Missouri’s 2024 GDP (real) | Approximately **$356.7 billion** (USAFacts) – roughly **$353 billion** (BEA) |

Thus, the Kansas City Chiefs won the 2024 Super Bowl, they are based in Missou